# Natural Language Processing with Amazon Reviews

Developed for the second project of the Artificial Intelligence course

## Setup and Environment

After installing Python and the necessary packages, run the following code to import them:

In [2]:
%matplotlib inline

import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import *
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

from sklearn.model_selection import PredefinedSplit
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

MAX_FEATURES = 10000

## Implementation work

### Generating a data set
#### Obtaining features from the Bag-of-Words model

In [3]:
train = pd.read_csv('./data/train_preprocessed.csv')
test = pd.read_csv('./data/test_preprocessed.csv')

Please choose one of the following vectorizers:
#### Creating regular CountVectorizers (default)

In [4]:
titleVectorizer = CountVectorizer(max_features=MAX_FEATURES)
textVectorizer = CountVectorizer(max_features=MAX_FEATURES)

#### Creating 1-hot vectors

In [ ]:
titleVectorizer = CountVectorizer(binary=True, max_features=MAX_FEATURES)
textVectorizer = CountVectorizer(binary=True, max_features=MAX_FEATURES)

#### Creating TF-IDF vectors

In [ ]:
titleVectorizer = TfidfVectorizer(max_features=MAX_FEATURES)
textVectorizer = TfidfVectorizer(max_features=MAX_FEATURES)

#### Creating bi-gram CountVectorizors

In [3]:
titleVectorizer = CountVectorizer(ngram_range=(1, 2), max_features=MAX_FEATURES)
textVectorizer = CountVectorizer(ngram_range=(1, 2), max_features=MAX_FEATURES)

#### Creating bi-gram TF-IDF vectors

In [ ]:
titleVectorizer = TfidfVectorizer(ngram_range=(1, 2), max_features=MAX_FEATURES)
textVectorizer = TfidfVectorizer(ngram_range=(1, 2), max_features=MAX_FEATURES)

#### Creating tri-gram CountVectorizors

In [ ]:
titleVectorizer = CountVectorizer(ngram_range=(1, 3), max_features=MAX_FEATURES)
textVectorizer = CountVectorizer(ngram_range=(1, 3), max_features=MAX_FEATURES)

#### Creating tri-gram TF-IDF vectors

In [ ]:
titleVectorizer = TfidfVectorizer(ngram_range=(1, 3), max_features=MAX_FEATURES)
textVectorizer = TfidfVectorizer(ngram_range=(1, 3), max_features=MAX_FEATURES)

#### Transforming the dataset

In [5]:
trainTitleArr = list(map(str, train['review_title']))
trainTextArr = list(map(str, train['review_text']))

titleWords = titleVectorizer.fit_transform(trainTitleArr).toarray()
textWords = textVectorizer.fit_transform(trainTextArr).toarray()

allWords = np.array([np.concatenate((titleWords[i], textWords[i])) for i in range(len(titleWords))])

print(titleWords.shape)
print(textWords.shape)
print(allWords.shape)

y = train['class_index']

# Test variables
testTitleArr = list(map(str, test['review_title']))
testTextArr = list(map(str, test['review_text']))

testTitleVector = titleVectorizer.transform(testTitleArr).toarray()
testTextVector = textVectorizer.transform(testTextArr).toarray()

testAllWords = np.array([np.concatenate((testTitleVector[i], testTextVector[i])) for i in range(len(testTitleVector))])

(5000, 4326)
(5000, 10000)
(5000, 14326)


### Prediction after searching parameters in grid

In [6]:
#param_grid = {"alpha": [0.005, 0.01, 0.05], "penalty": ['l2', 'l1', 'elasticnet', None]}
def gridPredict(clf, param_grid, **arg):
    grid = GridSearchCV(clf, param_grid, cv=2) # If we want Cross Validation: cv=pdfsplt

    grid.fit(allWords, y, **arg)

    print("Best grid params:", grid.best_params_)

    return grid.predict(testAllWords)

### Training Classifiers and Prediction

In [7]:
def predict(clf, **arg):
    # Train with allWords
    clf.fit(allWords, y, **arg)

    return clf.predict(testAllWords)

# MultinomialNB()

### Analyse Results

In [8]:
def analyze(prediction, test):
    y_test = test['class_index']

    # confusion matrix
    confusionMatrix = confusion_matrix(y_test, prediction)
    print("Confusion Matrix:")
    print(confusionMatrix)

    accuracy = accuracy_score(y_test, prediction)

    print("Accuracy: ", accuracy * 100, " %")
    print("Precision: ", precision_score(y_test, prediction, average='weighted') * 100, " %")
    print('Recall: ', recall_score(y_test, prediction, average='weighted') * 100, " %")
    print('F1: ', f1_score(y_test, prediction, average='weighted') * 100, " %")

### K-Nearest Neighbor

In [21]:
#prediction = predict(KNeighborsClassifier(n_neighbors=5))
prediction = gridPredict(KNeighborsClassifier(), {"n_neighbors": [5, 15, 30, 40], "leaf_size": [10, 20, 30, 35]})
print("K-Nearest Neighbor Results:")
analyze(prediction, test)

Best grid params: {'leaf_size': 10, 'n_neighbors': 40}
K-Nearest Neighbor Results:
Confusion Matrix:
[[ 54  17  10  83  36]
 [ 29  30  17  97  27]
 [ 18  11  11 118  42]
 [  7  10   6 121  56]
 [  7   3   9  93  88]]
Accuracy:  30.4  %
Precision:  33.78778756177409  %
Recall:  30.4  %
F1:  27.661714033918283  %


### Multinomial Naive Bays

In [8]:
# prediction = predict(MultinomialNB(), sample_weight=1.0)
prediction = gridPredict(MultinomialNB(), {"alpha": [0.0001, 0.5, 1.0]}, sample_weight=1.0)
print("MultinomialNB Results:")
analyze(prediction, test)

Best grid params: {'alpha': 1.0}
MultinomialNB Results:
Confusion Matrix:
[[106  62  17   7   8]
 [ 47  83  48  17   5]
 [ 20  72  54  36  18]
 [ 13  25  42  66  54]
 [  5  14  19  52 110]]
Accuracy:  41.9  %
Precision:  42.28163305892051  %
Recall:  41.9  %
F1:  41.93226954097882  %


### Random Forest

In [ ]:
# prediction = predict(RandomForestClassifier(n_estimators=100), sample_weight=1.0) 
# "max_features": ["int", "sqrt", "log2"]
prediction = gridPredict(RandomForestClassifier(), {"n_estimators": [50, 100]} , sample_weight=1.0)
# TODO: TEST SINGLE PARAMETER (n_estimators = 200 ...)
print("RandomForestClassifier Results:")
analyze(prediction, test)

### Decision Tree

In [22]:
#prediction = predict(DecisionTreeClassifier(), sample_weight=1.0)
prediction = gridPredict(DecisionTreeClassifier(), {"criterion": ["gini", "entropy", "log_loss"], "splitter": ["best", "random"]} , sample_weight=1.0)
print("DecisionTreeClassifier Results:")
analyze(prediction, test)

/home/bruno/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
4 fits failed out of a total of 12.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
4 fits failed with the following error:
Traceback (most recent call last):
  File "/home/bruno/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/bruno/anaconda3/lib/python3.9/site-packages/sklearn/tree/_classes.py", line 937, in fit
    super().fit(
  File "/home/bruno/anaconda3/lib/python3.9/site-packages/sklearn/tree/_classes.py", line 352, in fit
    criterion = CRITERIA_CLF[self.criterion](
KeyError: 'log_loss'

  warning

Best grid params: {'criterion': 'gini', 'splitter': 'best'}
DecisionTreeClassifier Results:
Confusion Matrix:
[[83 53 22 23 19]
 [55 43 47 33 22]
 [34 41 54 46 25]
 [17 23 49 58 53]
 [13 24 18 48 97]]
Accuracy:  33.5  %
Precision:  33.13434991037683  %
Recall:  33.5  %
F1:  33.289532259516896  %


### Perceptron

In [23]:
# prediction = predict(Perceptron(), sample_weight=1.0)
prediction = gridPredict(Perceptron(), {"alpha": [0.0001, 0.001]} , sample_weight=1.0)
print("Perceptron Results:")
analyze(prediction, test)

Best grid params: {'alpha': 0.0001}
Perceptron Results:
Confusion Matrix:
[[109  45  27   9  10]
 [ 51  80  30  30   9]
 [ 27  49  53  49  22]
 [ 21  16  27  83  53]
 [ 13  14  17  48 108]]
Accuracy:  43.3  %
Precision:  42.86348551441378  %
Recall:  43.3  %
F1:  42.935683522677394  %


### Logistic Regression

In [10]:
prediction = predict(LogisticRegression(), sample_weight=1.0)
# Best grid params: {'class_weight': 'balanced', 'solver': 'saga'} este foi o resultado do grid search
#prediction = gridPredict(LogisticRegression(), {#"penalty": ["l1", "l2", "elastic", "none"],
#    "class_weight": ["balanced", None], "solver": ["newton-cg", "lbfgs", "sag", "saga"]
#}, sample_weight=1.0)
print("LogisticRegression Results:")
analyze(prediction, test)

LogisticRegression Results:
Confusion Matrix:
[[121  41  22   7   9]
 [ 52  76  37  27   8]
 [ 28  48  60  43  21]
 [ 15  10  39  85  51]
 [  6  15  16  43 120]]
Accuracy:  46.2  %
Precision:  45.57338914038485  %
Recall:  46.2  %
F1:  45.8121814558217  %


/home/bruno/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### Linear SVC

In [15]:
prediction = predict(LinearSVC(), sample_weight=1.0)
# Best grid params: {'C': 0.2, 'class_weight': 'balanced', 'dual': True, 'loss': 'squared_hinge', 'multi_class': 'ovr'}
#prediction = gridPredict(LinearSVC(), {
#    "class_weight": ["balanced", None], "loss": ["hinge", "squared_hinge"], "multi_class": ["ovr", "crammer_singer"], "C": [0.2, 0.5, 1.0, 2.0], "dual": [True, False]
#}, sample_weight=1.0)
print("LinearSVC Results:")
analyze(prediction, test)

/home/bruno/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/bruno/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/bruno/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/bruno/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/bruno/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/bruno/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinea

Best grid params: {'C': 0.2, 'class_weight': 'balanced', 'dual': True, 'loss': 'squared_hinge', 'multi_class': 'ovr'}
LinearSVC Results:
Confusion Matrix:
[[114  46  25   6   9]
 [ 55  71  35  29  10]
 [ 30  54  54  40  22]
 [ 14  10  37  82  57]
 [  9  11  17  44 119]]
Accuracy:  44.0  %
Precision:  43.2216209477584  %
Recall:  44.0  %
F1:  43.51456960807654  %


### Neural Network (Multi-Layer Perceptron)

In [ ]:
prediction = predict(MLPClassifier())
print("NeuralNetwork Results:")
analyze(prediction, test)